In [2]:
# import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime

In [5]:
# create random latitude and longitude arrays
lats = np.random.uniform(-90.00, 90.000, size=2000)
longs = np.random.uniform(-180.00, 180.000, size=2000)

# zip to join lats and longs as tuple pairs
lat_lngs = zip(lats, longs)
lat_lngs

In [6]:
# unzip the tuples into a list
coordinates = list(lat_lngs)

In [13]:
# use citipy database to find closest city to each random lat and long pair

# initialize list to store cities
cities = []

# loop through coordinates and apply citipy functions
for geo in coordinates:
    city = citipy.nearest_city(geo[0], geo[1]).city_name

    # check to see if city already exists in cities list. If it isn't add it. 
    if city not in cities:
        cities.append(city)

# check length of cities to make sure we have enough datapoints
len(cities)

774

In [14]:
cities[:5]

['mokolo', 'marion', 'severo-kurilsk', 'alta floresta', 'krasnoselkup']

In [16]:
# API call to OpenWeatherMap to collect...
# longitude
# latitude
# maximum temperature
# percent humidity
# percent cloudiness
# wind speed
# weather description (cloud, fog, rain, etc.)

# creating url for parsing
openWeatherURL = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [26]:
# initialize empty list for city data - will hold dictionaries with city and datapoints
city_data = []

# create counters to iterate through cities
record_count = 1
set_count = 1

# enumerate cities list for index and city name
for i, city in enumerate(cities):

    # Group the cities in sets of 50 for logging and debugging
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    
    # set url for each city in cities
    city_url = f"{openWeatherURL}&q={city.replace(' ', '+')}"

    # log the URL, record, and set numbers of each city
    print(f"Processing {record_count} of Set {set_count} | {city}")
    # add 1 to our record counter
    record_count += 1

    # try API call and parse JSON
    try:
        city_weather = requests.get(city_url).json()

        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weath_desc = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]

        # if API call works append information to city_data in dictionary format
        city_data.append({"City": city.title(),
                        "Country": city_country,
                        "Lat": city_lat,
                        "Lng": city_lng,
                        "Max Temp": city_max_temp,
                        "Humidty": city_humidity,
                        "Cloudiness": city_clouds,
                        "Wind Speed": city_wind,
                        "Current Description": city_weath_desc
        })

        # If we don't have the information skip the city and move on 
    except:
        print("City not found. Skipping...")
        pass

# confirm loop has completed
print(f"--------------------\n Data Retrieval Complete")

7 of Set 5 | koronowo
Processing 28 of Set 5 | mys shmidta
City not found. Skipping...
Processing 29 of Set 5 | san luis
Processing 30 of Set 5 | ventspils
Processing 31 of Set 5 | hami
Processing 32 of Set 5 | vaitupu
City not found. Skipping...
Processing 33 of Set 5 | mildura
Processing 34 of Set 5 | luanda
Processing 35 of Set 5 | guerrero negro
Processing 36 of Set 5 | los llanos de aridane
Processing 37 of Set 5 | ilulissat
Processing 38 of Set 5 | louisbourg
City not found. Skipping...
Processing 39 of Set 5 | kamenskoye
City not found. Skipping...
Processing 40 of Set 5 | praya
Processing 41 of Set 5 | beringovskiy
Processing 42 of Set 5 | aksu
Processing 43 of Set 5 | yanan
City not found. Skipping...
Processing 44 of Set 5 | domokos
Processing 45 of Set 5 | sakakah
Processing 46 of Set 5 | batagay-alyta
Processing 47 of Set 5 | lagoa
Processing 48 of Set 5 | port macquarie
Processing 49 of Set 5 | emmauss
Processing 50 of Set 5 | puerto lleras
Processing 1 of Set 6 | sembakun

In [33]:
# Add processed data to DataFrame and export to .csv
city_data_df = pd.DataFrame(city_data)
city_data_df.to_csv("WeatherPy_Database.csv", index_label="City ID")